In [ ]:
# Install necessary packages in Google Colab
!pip install pathway bokeh --quiet


In [ ]:
import pandas as pd
import pathway as pw
import datetime
import pathway.debug as pw_debug


In [ ]:
# Model 3: Competitive Pricing

class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficScore: float
    IsSpecialDay: int
    VehicleWeight: float
    CompetitorAvgPrice: float

data = pw.demo.replay_csv("parking_stream_model3.csv", schema=ParkingSchema, input_rate=1000)

fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

alpha, beta, gamma, delta, epsilon, lambda_ = 2.0, 0.5, 1.5, 2.0, 1.0, 0.8
base_price = 10.0

delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        queue_max=pw.reducers.max(pw.this.QueueLength),
        traffic_max=pw.reducers.max(pw.this.TrafficScore),
        special_max=pw.reducers.max(pw.this.IsSpecialDay),
        vehicle_max=pw.reducers.max(pw.this.VehicleWeight),
        comp_price=pw.reducers.max(pw.this.CompetitorAvgPrice),
        cap=pw.reducers.max(pw.this.Capacity),
    )
    .with_columns(
        demand=(
            alpha * (pw.this.occ_max / pw.this.cap) +
            beta * pw.this.queue_max -
            gamma * pw.this.traffic_max +
            delta * pw.this.special_max +
            epsilon * pw.this.vehicle_max
        ),
        norm_demand=(pw.this.demand - 0) / 10,
        base_raw_price=base_price * (1 + lambda_ * pw.this.norm_demand),
        price=pw.this.base_raw_price.map(lambda x: max(min(x * 0.95 if x > pw.this.comp_price else x * 1.05, 20), 5))
    )
)

pw_debug.print_table(delta_window)
pw.run()
